# Setup

## Install Dependencies

In [1]:
!pip3 install ./pal

Processing ./pal
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pal: filename=pal-0.1-py3-none-any.whl size=19902 sha256=ce41d153cbb418147806e6c27088187c1ac46faecfcf76ca6f1800922a23f0b9
  Stored in directory: /private/var/folders/8d/b6tnc2p90l548qg78nc_k0_m0000gn/T/pip-ephem-wheel-cache-kixmatwr/wheels/fc/96/b9/2d39b0564063a661921e159ff6fdb50d8eb7e8bdd95704e9bf
Successfully built pal
  Attempting uninstall: pal
    Found existing installation: pal 0.1
    Uninstalling pal-0.1:
      Successfully uninstalled pal-0.1


## Configure Openai API

In [2]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
# @param {type:"string"}
openai.api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4.1-nano"  # @param {type:"string"}

# Web of Lies

In [3]:
import copy
import json
import argparse
import tqdm
import re
import openai
import os
from dotenv import load_dotenv
from pal import interface
from pal.prompt import web_of_lies_prompt

In [4]:
DATA_PATH = "datasets/web_of_lies.json"
OUTPUT_PATH = "eval_results/web_of_lies_PAL_gpt-4.1-nano.jsonl"

In [5]:
examples = json.load(open(DATA_PATH))["examples"]
print(examples)

[{'input': 'Question: Sherrie tells the truth. Vernell says Sherrie tells the truth. Alexis says Vernell lies. Michaela says Alexis tells the truth. Elanor says Michaela tells the truth. Does Elanor tell the truth?', 'target': False}, {'input': 'Question: Kristian lies. Sherrie says Kristian lies. Delbert says Sherrie lies. Jerry says Delbert tells the truth. Shalonda says Jerry tells the truth. Does Shalonda tell the truth?', 'target': False}, {'input': 'Question: Vina tells the truth. Helene says Vina lies. Kandi says Helene tells the truth. Jamey says Kandi lies. Ka says Jamey lies. Does Ka tell the truth?', 'target': False}, {'input': 'Question: Christie tells the truth. Ka says Christie tells the truth. Delbert says Ka lies. Leda says Delbert tells the truth. Lorine says Leda tells the truth. Does Lorine tell the truth?', 'target': False}, {'input': 'Question: Tamika lies. Raymond says Tamika tells the truth. Willian says Raymond tells the truth. Shaunda says Willian lies. Elanor 

In [6]:
itf = interface.ProgramInterface(
    stop="\n\n\n", get_answer_symbol="answer", verbose=True
)

In [7]:
append = False

In [8]:
if append:
    lines = open(OUTPUT_PATH).readlines()
    num_skip_exps = len(lines)
    scores = [x["score"] for x in map(json.loads, lines)]
else:
    num_skip_exps = 0
    scores = []

with open(OUTPUT_PATH, "a" if append else "w") as f:
    pbar = tqdm.tqdm(
        examples[num_skip_exps:], initial=num_skip_exps, total=len(examples)
    )
    for x in pbar:
        question = x["input"]
        result = copy.copy(x)
        try:
            # print(
            #     web_of_lies_prompt.WEB_OF_LIES_PROMPT.format(
            #         question=question
            #     )
            # )
            ans = itf.run(
                web_of_lies_prompt.WEB_OF_LIES_PROMPT.format(question=question)
            )
            
            # ans = True
            target = x["target"]
            score = int(ans == target)  # 1 if ans matches target, 0 otherwise

            result["score"] = score
            result["answer"] = ans

        except Exception as e:
            result["error"] = str(e)
            score = 0

        scores.append(score)

        result["score"] = score
        # result['generation'] = itf.history
        f.write(json.dumps(result) + "\n")

        itf.clear_history()
        f.flush()
print(f"Accuracy - {sum(scores) / len(scores)}")

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 1/250 [00:03<15:20,  3.70s/it]

['Sherrie = True\nVernell_says_Sherrie = True\nAlexis_says_Vernell = False\nMichaela_says_Alexis = True\nElanor_says_Michaela = True\nVernell = Vernell_says_Sherrie == Sherrie\nAlexis = Alexis_says_Vernell == Vernell\nMichaela = Michaela_says_Alexis == Alexis\nElanor = Elanor_says_Michaela == Michaela\nanswer = Elanor']


  1%|          | 2/250 [00:06<13:05,  3.17s/it]

['Kristian = False\nSherrie_says_Kristian = False\nDelbert_says_Sherrie = False\nJerry_says_Delbert = True\nShalonda_says_Jerry = True\nSherrie = Sherrie_says_Kristian == Kristian\nDelbert = Delbert_says_Sherrie == Sherrie\nJerry = Jerry_says_Delbert == Delbert\nShalonda = Shalonda_says_Jerry == Jerry\nanswer = Shalonda']


...

['Kandi = True\nRyan_says_Kandi = False\nMichaela_says_Ryan = True\nSima_says_Michaela = False\nAndree_says_Sima = True\nRyan = Ryan_says_Kandi == Kandi\nMichaela = Michaela_says_Ryan == Ryan\nSima = Sima_says_Michaela == Michaela\nAndree = Andree_says_Sima == Sima\nanswer = Andree']


100%|█████████▉| 249/250 [20:49<00:04,  4.88s/it]

['Fletcher = True\nAlexis_says_Fletcher = False\nJerry_says_Alexis = False\nElanor_says_Jerry = True\nJim_says_Elanor = False\nAlexis = Alexis_says_Fletcher == Fletcher\nJerry = Jerry_says_Alexis == Alexis\nElanor = Elanor_says_Jerry == Jerry\nJim = Jim_says_Elanor == Elanor\nanswer = Jim']


100%|██████████| 250/250 [20:55<00:00,  5.02s/it]

['Sherrie = True\nRyan_says_Sherrie = False\nBernita_says_Ryan = True\nTamika_says_Bernita = False\nJerry_says_Tamika = False\nRyan = Ryan_says_Sherrie == Sherrie\nBernita = Bernita_says_Ryan == Ryan\nTamika = Tamika_says_Bernita == Bernita\nJerry = Jerry_says_Tamika == Tamika\nanswer = Jerry']
Accuracy - 1.0


In [9]:
print(f"Accuracy - {sum(scores) / len(scores)}")

Accuracy - 1.0
